# Inferring Topics from IMDB Reviews

In [1]:
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pandas as pd
import matplotlib.pyplot as plt

## Exploring the Dataset: [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)

In [2]:
ROOT = './train/pos/'

In [3]:
reviews = []
for file in os.listdir(ROOT):
    path = os.path.join(ROOT, file)
    if os.path.isfile(path):
        with open(path, 'r') as fin:
            reviews.append(fin.read())

In [4]:
len(reviews)

2050

In [5]:
for i in range(3):
    print(reviews[i])
    print('=' * 150)

One of my favourite films first saw it when I was about 10, which probably tells you a lot about the type of humour. Although dated the humour definitely has a charm about it. Expect to see the usual Askey & Murdoch banter so popular in its day, with lots of interesting, quirky co-characters. The lady with the parrot, the couple due to get married and are in trouble from 'her', and my favourite, the stationmaster, "Nobody knows where it comes from ... nobody knows where it goes.." Interestingly the ghost train was written by Arnold Ridley of Dads Army fame (Private Godfrey the medic) Watch it on a rainy Sunday afternoon after your lunch and smile.
Having not seen the films before (and not being able to stand Matt Damon), I was reluctant to go see The Bourne Ultimatum when we were asked to see it for AS Film Studies. <br /><br />However, I was pleasantly surprised that even a film with Damon in it could be enjoyable. <br /><br />Fast fight scenes, crazy motorbike chases and BIG explosio

## Feature Extraction

In [6]:
vect = TfidfVectorizer(stop_words='english')
X = vect.fit_transform(reviews)

pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())

,00,000,007,0080,00s,01,02,03,05,06,...,álex,álvaro,ángel,äänekoski,åge,écran,émigrés,était,ís,østbye
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## NMF Decomposition

In [7]:
N_TOPICS = 15
nmf = NMF(n_components=N_TOPICS)
W = nmf.fit_transform(X)  # Document-topic matrix
H = nmf.components_       # Topic-term matrix

In [8]:
# Top 10 words per topic

words = np.array(vect.get_feature_names_out())
topic_words = pd.DataFrame(np.zeros((N_TOPICS, 10)), index=[f'Topic {i + 1}' for i in range(N_TOPICS)],
                           columns=[f'Word {i + 1}' for i in range(10)]).astype(str)
for i in range(N_TOPICS):
    ix = H[i].argsort()[::-1][:10]
    topic_words.iloc[i] = words[ix]

topic_words

,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10
Topic 1,br,10,scene,plot,ve,ll,high,story,just,quite
Topic 2,movie,movies,saw,watch,fun,recommend,acting,seen,watching,thought
Topic 3,film,films,seen,style,director,watch,character,characters,acting,like
Topic 4,good,really,just,like,think,don,say,know,pretty,didn
Topic 5,life,people,love,world,young,story,time,real,lives,like
Topic 6,man,role,john,character,cast,performance,mr,best,actor,murder
Topic 7,funny,comedy,eddie,hilarious,funniest,time,laugh,10,murphy,seen
Topic 8,series,episode,tv,episodes,war,dvd,television,new,trek,sci
Topic 9,tom,jerry,cartoon,invisible,short,mouse,cat,cartoons,butch,lee
Topic 10,disney,cinderella,animation,holes,fairy,mice,king,lion,ball,prince


In [9]:
# Create a topic mapping

topic_mapping = {
    'Topic 4': 'TV',
    'Topic 7': 'War',
    'Topic 8': 'Comedy',
    'Topic 12': 'Book Adaptation',
    'Topic 13': 'Horror',
    'Topic 15': 'Martial Arts / Action'
}

In [10]:
# Recall the document-topic matrix, W

W = pd.DataFrame(W, columns=[f'Topic {i + 1}' for i in range(N_TOPICS)])
W['max_topic'] = W.apply(lambda x: topic_mapping.get(x.idxmax()), axis=1)
W[pd.notnull(W['max_topic'])].head(10)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,max_topic
0,0.001709,0.005090,0.010125,0.006606,0.009450,0.005346,0.019238,0.000511,0.004539,0.000851,0.009258,0.019805,0.010484,0.000000,0.007128,Book Adaptation
5,0.000000,0.029897,0.004220,0.000000,0.025900,0.000000,0.004021,0.017535,0.000000,0.000000,0.000000,0.142740,0.000000,0.031049,0.000000,Book Adaptation
12,0.021187,0.003621,0.003097,0.025888,0.016486,0.000000,0.000127,0.006874,0.004127,0.000000,0.003302,0.000000,0.000000,0.000000,0.000000,TV
22,0.000000,0.030251,0.018309,0.000241,0.000000,0.000000,0.049884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003907,War
23,0.000000,0.011792,0.032559,0.097721,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078554,0.000000,0.004167,0.000000,TV
29,0.019033,0.002670,0.000000,0.008138,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.354976,0.000000,0.000000,Horror
31,0.000000,0.018644,0.048057,0.002984,0.002053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.177354,0.000000,0.015532,0.000000,Book Adaptation
38,0.045763,0.020941,0.053219,0.019736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.201999,0.009646,0.000000,0.013213,Book Adaptation
55,0.000403,0.020196,0.000000,0.013384,0.000000,0.005359,0.037717,0.000000,0.000000,0.000000,0.014259,0.000000,0.006027,0.033681,0.000658,War
61,0.000035,0.000916,0.002000,0.000000,0.000000,0.000000,0.013017,0.000000,0.000000,0.000000,0.002144,0.063545,0.000000,0.001415,0.000000,Book Adaptation


In [11]:
reviews[58]

'James Dickey is a wonderfully descriptive author. When one reads "Deliverance", one is instantly transported into the lush backwoods of the Deep South. When one watches John Boorman\'s film version of the book, one realizes just how accurately he captures the essence of the book. The camera is as descriptive as the narration. The characters are fully realized, and the portrayals are fantastic. I first saw this movie in 1992, after my freshman year of college. I was in a phase where I was watching movies that were all released within a couple of years of my birth in 1973. Among them were "Patton", "Papillon", and "All the President\'s Men"; fine films, all of them. This one was easily the class of the group. That says a lot.'

In [12]:
# Frobenius norm

import numpy as np

print("Frobenius norm and the condition number:")
print(np.linalg.norm([[1,1,1],[3,4,1],[4,1,2]], 'fro'))
print(np.linalg.cond([[1,1,1],[3,4,1],[4,1,2]], 'fro'))


Frobenius norm and the condition number:
7.0710678118654755
13.975424859373685


In [13]:
import datetime, pytz; 
print("Current Time in IST:", datetime.datetime.now(pytz.utc).astimezone(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S'))

Current Time in IST: 2025-02-12 17:04:46
